Classificação de Texto Multilabel

José Augusto de Almeida Neto

Modelo BERT pré treinado: [BERTimbau Base](https://huggingface.co/neuralmind/bert-base-portuguese-cased)

Referências:

https://www.youtube.com/watch?v=f-86-HcYYi8

https://github.com/theartificialguy/NLP-with-Deep-Learning/blob/master/BERT/Multi%20Label%20Text%20Classification%20using%20BERT%20PyTorch/bert_multilabel_pytorch_standard.ipynb

https://www.kaggle.com/code/pritishmishra/fine-tune-bert-for-text-classification?scriptVersionId=116951029

https://medium.com/analytics-vidhya/multi-label-text-classification-using-transformers-bert-93460838e62b

https://huggingface.co/docs/transformers/v4.39.3/en/model_doc/bert#transformers.BertForSequenceClassification

# Preparação antes da aplicação do modelo

## Importar bibliotecas

In [ ]:
import re
import time
import tensorflow as tf
import torch
import pandas as pd
import nltk
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score, classification_report)
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, random_split
from sklearn.preprocessing import MultiLabelBinarizer

## Carregar datasets


Datasets com Stemming
```
file_urls = {
    'df_fold1': 'https://drive.google.com/uc?export=download&id=1x-emOdLX_1zfc1W0DYw5ZE8fJGTYG90E',
    'df_fold2': 'https://drive.google.com/uc?export=download&id=1wyTiXHpqVdNGgLeYLiXXIauxhkWIoI09',
    'df_fold3': 'https://drive.google.com/uc?export=download&id=1HlTxTZwk89lUFahsBVnbP8KcWCXt0ajp',
    'df_fold4': 'https://drive.google.com/uc?export=download&id=1ECUpYnYJXPdF02m4MPAScv8CB8ms2JlG',
    'df_fold5': 'https://drive.google.com/uc?export=download&id=1d3icXhLKnzQnmU81clYOrc7q0UxTipAp'
}
```
Datasets com Spacy
```
file_urls = {
    'df_fold1': 'https://drive.google.com/uc?export=download&id=1hZMkMcOoo2UzMvDb7k9vfG49rG0pCdrU',
    'df_fold2': 'https://drive.google.com/uc?export=download&id=11UL95ELTzg1ewknf8tvI_c8m2Ur4xm9h',
    'df_fold3': 'https://drive.google.com/uc?export=download&id=1I0hP1UAUHz-avgZu1m6QWQVw4x-3ItlE',
    'df_fold4': 'https://drive.google.com/uc?export=download&id=1jCbvDUruEav365DlVMMrslou2ws6ArF0',
    'df_fold5': 'https://drive.google.com/uc?export=download&id=1EJsZYSsRoB8tpkd13p_Bkc_XBeKMb5Em'
}
```
Datasets com Lemmatization
```
file_urls = {
    'df_fold1': 'https://drive.google.com/uc?export=download&id=1ATF0g4uRFXwg546SB_zwlFE_rbivQdM1',
    'df_fold2': 'https://drive.google.com/uc?export=download&id=1LTonHJQVHUq-utKl57LjZic58skyje0z',
    'df_fold3': 'https://drive.google.com/uc?export=download&id=1dXxLLWK4gFLeW-mJJ4DQ4hlwEJdkErBO',
    'df_fold4': 'https://drive.google.com/uc?export=download&id=11OR2kwSl3WinMcF3YqD3xCPj8ESo2Tba',
    'df_fold5': 'https://drive.google.com/uc?export=download&id=1_Rg4zwANXPwDTtVUKkA26jwf2F1l6FbY'
}
```
Datasets com Stopwords
```
file_urls = {
    'df_fold1': 'https://drive.google.com/uc?export=download&id=11Uect4s84pU5pSei5tRazA3hwavJx-Nh',
    'df_fold2': 'https://drive.google.com/uc?export=download&id=1sJ00Z-ynBsercqwgCNLfDNZKkvvB5G8R',
    'df_fold3': 'https://drive.google.com/uc?export=download&id=1Imvb4D8IgnKnXCGBc2PMr6HOLzxjRPd6',
    'df_fold4': 'https://drive.google.com/uc?export=download&id=1-ebIQ_DCCvTnJUyLONHYqMkLige_vdWs',
    'df_fold5': 'https://drive.google.com/uc?export=download&id=10XHVDgB8r-n25xYZcRFPoDYTH61CyFOC'
}
```


In [ ]:
# URLs dos datasets
file_urls = {
    'df_fold1': 'https://drive.google.com/uc?export=download&id=1x-emOdLX_1zfc1W0DYw5ZE8fJGTYG90E',
    'df_fold2': 'https://drive.google.com/uc?export=download&id=1wyTiXHpqVdNGgLeYLiXXIauxhkWIoI09',
    'df_fold3': 'https://drive.google.com/uc?export=download&id=1HlTxTZwk89lUFahsBVnbP8KcWCXt0ajp',
    'df_fold4': 'https://drive.google.com/uc?export=download&id=1ECUpYnYJXPdF02m4MPAScv8CB8ms2JlG',
    'df_fold5': 'https://drive.google.com/uc?export=download&id=1d3icXhLKnzQnmU81clYOrc7q0UxTipAp'
}

def load_dataset(url):
    return pd.read_csv(url)

# Carregar datasets
df_fold1 = load_dataset(file_urls['df_fold1'])
df_fold2 = load_dataset(file_urls['df_fold2'])
df_fold3 = load_dataset(file_urls['df_fold3'])
df_fold4 = load_dataset(file_urls['df_fold4'])
df_fold5 = load_dataset(file_urls['df_fold5'])

In [ ]:
labels = ['ambiente', 'bebida', 'comida', 'geral', 'localização', 'outros', 'preço', 'serviço']

In [ ]:
def remove_empty_sentenca_rows(df):
    empty_sentenca_rows = df['sentenca'].isnull() | (df['sentenca'] == '')
    df = df[~empty_sentenca_rows]
    df.reset_index(drop=True, inplace=True)
    return df

# Removendo linhas vazias
df_fold1 = remove_empty_sentenca_rows(df_fold1)
df_fold2 = remove_empty_sentenca_rows(df_fold2)
df_fold3 = remove_empty_sentenca_rows(df_fold3)
df_fold4 = remove_empty_sentenca_rows(df_fold4)
df_fold5 = remove_empty_sentenca_rows(df_fold5)

## Histórico de resultados - Fold 1 (Hiperparâmetros)

### Stemming

Teste 4
```
# Hiperparâmetros
max_len = 128
train_batch_size = 8
valid_batch_size = 8
epochs = 5
learning_rate = 1e-05
num_labels = len(labels)
threshold = 0.3
```
* Tempo de execução: 4h18m34s
* Epoch 5:
  - Macro F1-score: 0.9307
  - Micro F1-score: 0.9366



Teste 8
```
# Hiperparâmetros
max_len = 128
train_batch_size = 16
valid_batch_size = 8
epochs = 5 (3)
learning_rate = 2e-05
num_labels = len(labels)
threshold = 0.35
```
* Tempo de execução: 5h28m46s
* Epoch 3:
  - Macro F1-score: 0.9308
  - Micro F1-score: 0.9333

### Spacy + pontuação + strip

Teste 2
```
# Hiperparâmetros
max_len = 128
train_batch_size = 16
valid_batch_size = 8
epochs = 5
learning_rate = 2e-05
num_labels = len(labels)
threshold = 0.35
```
* Tempo de execução: 4h16m4s
* Epoch 4:
  - Macro F1-score: 0.8906
  - Micro F1-score: 0.9006

Teste 6
```
# Hiperparâmetros
max_len = 128
train_batch_size = 16
valid_batch_size = 8
epochs = 7
learning_rate = 2e-05
num_labels = len(labels)
threshold = 0.35
```
* Tempo de execução: 6h39m35s
* Epoch 4:
  - Macro F1-score: 0.8943
  - Micro F1-score: 0.9050

### Lemmatization + lower + pontuação + strip

Teste 2
```
# Hiperparâmetros
max_len = 128
train_batch_size = 8
valid_batch_size = 8
epochs = 5
learning_rate = 1e-05
num_labels = len(labels)
threshold = 0.3
```
* Tempo de execução: 6h33m36s
* Epoch 5:
  - Macro F1-score: 0.9337
  - Micro F1-score: 0.9336

Teste 5
```
# Hiperparâmetros
max_len = 128
train_batch_size = 8
valid_batch_size = 8
epochs = 6
learning_rate = 2e-05
num_labels = len(labels)
threshold = 0.25
```
* Tempo de execução: 4h59m0s
* Epoch 6:
  - Macro F1-score: 0.9302
  - Micro F1-score: 0.9311

Teste 9
```
# Hiperparâmetros
max_len = 128
train_batch_size = 12
valid_batch_size = 8
epochs = 6
learning_rate = 2.5e-05
num_labels = len(labels)
threshold = 0.3
```
* Tempo de execução: 2h27m38s
* Epoch 4:
  - Macro F1-score: 0.9283
  - Micro F1-score: 0.9343

### Stopwords + pontuação + strip

Teste 2
```
# Hiperparâmetros
max_len = 128
train_batch_size = 16
valid_batch_size = 8
epochs = 5
learning_rate = 2e-05
num_labels = len(labels)
threshold = 0.35
```
* Tempo de execução: 6h13m37s
* Epoch 4:
  - Macro F1-score: 0.9223
  - Micro F1-score: 0.9252

Teste 3
```
# Hiperparâmetros
max_len = 128
train_batch_size = 8
valid_batch_size = 8
epochs = 5
learning_rate = 3e-05
num_labels = len(labels)
threshold = 0.3
```
* Tempo de execução: 6h28m12s
* Epoch 3:
  - Macro F1-score: 0.9250
  - Micro F1-score: 0.9251

## Listas, hiperparâmetros e tokenização

In [ ]:
# Gerar listas de métricas
f1_score_list_macro = []
precision_list_macro = []
recall_list_macro = []
f1_score_list_micro = []
precision_list_micro = []
recall_list_micro = []

In [ ]:
# Hiperparâmetros
max_len = 128
train_batch_size = 16
valid_batch_size = 8
epochs = 3
learning_rate = 2e-05
num_labels = len(labels)
threshold = 0.35

In [ ]:
# Carregar o tokenizador BERT
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

In [ ]:
class TokenizeDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['sentenca']
        self.targets = self.df[labels].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

# BERT

## 1° Fold

In [ ]:
# Dividir o dataset
test_data = df_fold1
train_data = pd.concat([df_fold2, df_fold3, df_fold4, df_fold5], ignore_index=True)

In [ ]:
# Tokenizar o dataset
tokenized_train_data = TokenizeDataset(train_data, tokenizer, max_len)
tokenized_test_data = TokenizeDataset(test_data, tokenizer, max_len)

In [ ]:
# Criar DataLoader para carregar os dados em lotes durante o treinamento
train_loader = DataLoader(tokenized_train_data, batch_size=train_batch_size, shuffle=True)
val_loader = DataLoader(tokenized_test_data, batch_size=valid_batch_size, shuffle=False)

In [ ]:
# Definir modelo, optimizer, e loss function
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=num_labels, problem_type="multi_label_classification")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.BCEWithLogitsLoss()
sigmoid = torch.nn.Sigmoid()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
start_time = time.time()

for epoch in range(epochs):
    # Loop de Treinamento
    model.train() # modo de treinamento
    for batch in train_loader:
        inputs_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        targets = batch['targets']

        # Passagem Direta - os dados de entrada são passados para o modelo
        outputs = model(input_ids=inputs_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        # A função sigmóide é aplicada aos logits para converter os valores em probabilidades
        logits = outputs.logits
        predicted_probs = sigmoid(logits)

        # A perda é calculada comparando os logits com os rótulos verdadeiros (targets)
        loss = criterion(logits, targets) # função de perda criterion

        # Retropropagação e Otimização
        optimizer.zero_grad()
        loss.backward() #  gradiente é calculado
        optimizer.step() # parâmetros do modelo são otimizados

    # Loop de Validação
    model.eval() # modo de avaliação
    all_targets = []
    all_predictions = []
    with torch.no_grad():
        for batch in val_loader:
            inputs_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            token_type_ids = batch['token_type_ids']
            targets = batch['targets']

            # Passagem Direta (inferência) - da mesma foram que o treinamento
            outputs = model(input_ids=inputs_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

            # A função sigmóide é aplicada aos logits para converter os valores em probabilidades
            logits = outputs.logits
            predicted_probs = sigmoid(logits)

            # Aplicação do threshold
            predicted_labels = (predicted_probs > threshold).float()

            # Conversão dos rótulos verdadeiros e previstos para numpy arrays
            targets_np = targets.cpu().numpy()
            predicted_labels_np = predicted_labels.cpu().numpy()

            # Coleção dos targets e predictions para calculas as métricas
            all_targets.extend(targets_np)
            all_predictions.extend(predicted_labels_np)

    # Calcular as métricas
    f1_macro = f1_score(all_targets, all_predictions, average='macro')
    precision_macro = precision_score(all_targets, all_predictions, average='macro')
    recall_macro = recall_score(all_targets, all_predictions, average='macro')
    f1_micro = f1_score(all_targets, all_predictions, average='micro')
    precision_micro = precision_score(all_targets, all_predictions, average='micro')
    recall_micro = recall_score(all_targets, all_predictions, average='micro')

    # Atualizar a lista das métricas apenas da última epoch
    if epoch == epochs - 1:
        f1_score_list_macro.append(f1_macro)
        precision_list_macro.append(precision_macro)
        recall_list_macro.append(recall_macro)
        f1_score_list_micro.append(f1_micro)
        precision_list_micro.append(precision_micro)
        recall_list_micro.append(recall_micro)

    # Imprimir as métricas ao final de cada época
    print(f"Epoch {epoch + 1}:")
    print(f"F1-Score (macro): {f1_macro:.4f}, Precision (macro): {precision_macro:.4f}, Recall (macro): {recall_macro:.4f}")
    print(f"F1-Score (micro): {f1_micro:.4f}, Precision (micro): {precision_micro:.4f}, Recall (micro): {recall_micro:.4f}")

# Calcular o tempo total de execução
end_time = time.time()
elapsed_time = end_time - start_time
hours, rem = divmod(elapsed_time, 3600)
minutes, seconds = divmod(rem, 60)

print(f"\nTempo de execução: {int(hours):02}:{int(minutes):02}:{seconds:.2f}")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1:
F1-Score (macro): 0.6518, Precision (macro): 0.6561, Recall (macro): 0.7131
F1-Score (micro): 0.8255, Precision (micro): 0.7573, Recall (micro): 0.9072


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2:
F1-Score (macro): 0.7978, Precision (macro): 0.7615, Recall (macro): 0.8410
F1-Score (micro): 0.9099, Precision (micro): 0.8689, Recall (micro): 0.9549
Epoch 3:
F1-Score (macro): 0.8773, Precision (macro): 0.9061, Recall (macro): 0.8888
F1-Score (micro): 0.9257, Precision (micro): 0.9001, Recall (micro): 0.9529
Epoch 4:
F1-Score (macro): 0.8920, Precision (macro): 0.8855, Recall (macro): 0.9132
F1-Score (micro): 0.9204, Precision (micro): 0.8883, Recall (micro): 0.9549
Epoch 5:
F1-Score (macro): 0.8849, Precision (macro): 0.8773, Recall (macro): 0.9094
F1-Score (micro): 0.9231, Precision (micro): 0.8898, Recall (micro): 0.9590

Tempo de execução: 05:46:22.67


## 2° Fold

In [ ]:
# Dividir o dataset
test_data = df_fold2
train_data = pd.concat([df_fold1, df_fold3, df_fold4, df_fold5], ignore_index=True)

In [ ]:
# Tokenizar o dataset
tokenized_train_data = TokenizeDataset(train_data, tokenizer, max_len)
tokenized_test_data = TokenizeDataset(test_data, tokenizer, max_len)

In [ ]:
# Criar DataLoader para carregar os dados em lotes durante o treinamento
train_loader = DataLoader(tokenized_train_data, batch_size=train_batch_size, shuffle=True)
val_loader = DataLoader(tokenized_test_data, batch_size=valid_batch_size, shuffle=False)

In [ ]:
# Definir modelo, optimizer, e loss function
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=num_labels, problem_type="multi_label_classification")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.BCEWithLogitsLoss()
sigmoid = torch.nn.Sigmoid()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
start_time = time.time()

for epoch in range(epochs):
    # Loop de Treinamento
    model.train() # modo de treinamento
    for batch in train_loader:
        inputs_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        targets = batch['targets']

        # Passagem Direta - os dados de entrada são passados para o modelo
        outputs = model(input_ids=inputs_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        # A função sigmóide é aplicada aos logits para converter os valores em probabilidades
        logits = outputs.logits
        predicted_probs = sigmoid(logits)

        # A perda é calculada comparando os logits com os rótulos verdadeiros (targets)
        loss = criterion(logits, targets) # função de perda criterion

        # Retropropagação e Otimização
        optimizer.zero_grad()
        loss.backward() #  gradiente é calculado
        optimizer.step() # parâmetros do modelo são otimizados

    # Loop de Validação
    model.eval() # modo de avaliação
    all_targets = []
    all_predictions = []
    with torch.no_grad():
        for batch in val_loader:
            inputs_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            token_type_ids = batch['token_type_ids']
            targets = batch['targets']

            # Passagem Direta (inferência) - da mesma foram que o treinamento
            outputs = model(input_ids=inputs_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

            # A função sigmóide é aplicada aos logits para converter os valores em probabilidades
            logits = outputs.logits
            predicted_probs = sigmoid(logits)

            # Aplicação do threshold
            predicted_labels = (predicted_probs > threshold).float()

            # Conversão dos rótulos verdadeiros e previstos para numpy arrays
            targets_np = targets.cpu().numpy()
            predicted_labels_np = predicted_labels.cpu().numpy()

            # Coleção dos targets e predictions para calculas as métricas
            all_targets.extend(targets_np)
            all_predictions.extend(predicted_labels_np)

    # Calcular as métricas
    f1_macro = f1_score(all_targets, all_predictions, average='macro')
    precision_macro = precision_score(all_targets, all_predictions, average='macro')
    recall_macro = recall_score(all_targets, all_predictions, average='macro')
    f1_micro = f1_score(all_targets, all_predictions, average='micro')
    precision_micro = precision_score(all_targets, all_predictions, average='micro')
    recall_micro = recall_score(all_targets, all_predictions, average='micro')

    # Atualizar a lista das métricas apenas da última epoch
    if epoch == epochs - 1:
        f1_score_list_macro.append(f1_macro)
        precision_list_macro.append(precision_macro)
        recall_list_macro.append(recall_macro)
        f1_score_list_micro.append(f1_micro)
        precision_list_micro.append(precision_micro)
        recall_list_micro.append(recall_micro)

    # Imprimir as métricas ao final de cada época
    print(f"Epoch {epoch + 1}:")
    print(f"F1-Score (macro): {f1_macro:.4f}, Precision (macro): {precision_macro:.4f}, Recall (macro): {recall_macro:.4f}")
    print(f"F1-Score (micro): {f1_micro:.4f}, Precision (micro): {precision_micro:.4f}, Recall (micro): {recall_micro:.4f}")

# Calcular o tempo total de execução
end_time = time.time()
elapsed_time = end_time - start_time
hours, rem = divmod(elapsed_time, 3600)
minutes, seconds = divmod(rem, 60)

print(f"\nTempo de execução: {int(hours):02}:{int(minutes):02}:{seconds:.2f}")

## 3° Fold

In [ ]:
# Dividir o dataset
test_data = df_fold3
train_data = pd.concat([df_fold1, df_fold2, df_fold4, df_fold5], ignore_index=True)

In [ ]:
# Tokenizar o dataset
tokenized_train_data = TokenizeDataset(train_data, tokenizer, max_len)
tokenized_test_data = TokenizeDataset(test_data, tokenizer, max_len)

In [ ]:
# Criar DataLoader para carregar os dados em lotes durante o treinamento
train_loader = DataLoader(tokenized_train_data, batch_size=train_batch_size, shuffle=True)
val_loader = DataLoader(tokenized_test_data, batch_size=valid_batch_size, shuffle=False)

In [ ]:
# Definir modelo, optimizer, e loss function
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=num_labels, problem_type="multi_label_classification")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.BCEWithLogitsLoss()
sigmoid = torch.nn.Sigmoid()

In [ ]:
start_time = time.time()

for epoch in range(epochs):
    # Loop de Treinamento
    model.train() # modo de treinamento
    for batch in train_loader:
        inputs_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        targets = batch['targets']

        # Passagem Direta - os dados de entrada são passados para o modelo
        outputs = model(input_ids=inputs_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        # A função sigmóide é aplicada aos logits para converter os valores em probabilidades
        logits = outputs.logits
        predicted_probs = sigmoid(logits)

        # A perda é calculada comparando os logits com os rótulos verdadeiros (targets)
        loss = criterion(logits, targets) # função de perda criterion

        # Retropropagação e Otimização
        optimizer.zero_grad()
        loss.backward() #  gradiente é calculado
        optimizer.step() # parâmetros do modelo são otimizados

    # Loop de Validação
    model.eval() # modo de avaliação
    all_targets = []
    all_predictions = []
    with torch.no_grad():
        for batch in val_loader:
            inputs_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            token_type_ids = batch['token_type_ids']
            targets = batch['targets']

            # Passagem Direta (inferência) - da mesma foram que o treinamento
            outputs = model(input_ids=inputs_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

            # A função sigmóide é aplicada aos logits para converter os valores em probabilidades
            logits = outputs.logits
            predicted_probs = sigmoid(logits)

            # Aplicação do threshold
            predicted_labels = (predicted_probs > threshold).float()

            # Conversão dos rótulos verdadeiros e previstos para numpy arrays
            targets_np = targets.cpu().numpy()
            predicted_labels_np = predicted_labels.cpu().numpy()

            # Coleção dos targets e predictions para calculas as métricas
            all_targets.extend(targets_np)
            all_predictions.extend(predicted_labels_np)

    # Calcular as métricas
    f1_macro = f1_score(all_targets, all_predictions, average='macro')
    precision_macro = precision_score(all_targets, all_predictions, average='macro')
    recall_macro = recall_score(all_targets, all_predictions, average='macro')
    f1_micro = f1_score(all_targets, all_predictions, average='micro')
    precision_micro = precision_score(all_targets, all_predictions, average='micro')
    recall_micro = recall_score(all_targets, all_predictions, average='micro')

    # Atualizar a lista das métricas apenas da última epoch
    if epoch == epochs - 1:
        f1_score_list_macro.append(f1_macro)
        precision_list_macro.append(precision_macro)
        recall_list_macro.append(recall_macro)
        f1_score_list_micro.append(f1_micro)
        precision_list_micro.append(precision_micro)
        recall_list_micro.append(recall_micro)

    # Imprimir as métricas ao final de cada época
    print(f"Epoch {epoch + 1}:")
    print(f"F1-Score (macro): {f1_macro:.4f}, Precision (macro): {precision_macro:.4f}, Recall (macro): {recall_macro:.4f}")
    print(f"F1-Score (micro): {f1_micro:.4f}, Precision (micro): {precision_micro:.4f}, Recall (micro): {recall_micro:.4f}")

# Calcular o tempo total de execução
end_time = time.time()
elapsed_time = end_time - start_time
hours, rem = divmod(elapsed_time, 3600)
minutes, seconds = divmod(rem, 60)

print(f"\nTempo de execução: {int(hours):02}:{int(minutes):02}:{seconds:.2f}")

## 4° Fold

In [ ]:
# Dividir o dataset
test_data = df_fold4
train_data = pd.concat([df_fold1, df_fold2, df_fold3, df_fold5], ignore_index=True)

In [ ]:
# Tokenizar o dataset
tokenized_train_data = TokenizeDataset(train_data, tokenizer, max_len)
tokenized_test_data = TokenizeDataset(test_data, tokenizer, max_len)

In [ ]:
# Criar DataLoader para carregar os dados em lotes durante o treinamento
train_loader = DataLoader(tokenized_train_data, batch_size=train_batch_size, shuffle=True)
val_loader = DataLoader(tokenized_test_data, batch_size=valid_batch_size, shuffle=False)

In [ ]:
# Definir modelo, optimizer, e loss function
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=num_labels, problem_type="multi_label_classification")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.BCEWithLogitsLoss()
sigmoid = torch.nn.Sigmoid()

In [ ]:
start_time = time.time()

for epoch in range(epochs):
    # Loop de Treinamento
    model.train() # modo de treinamento
    for batch in train_loader:
        inputs_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        targets = batch['targets']

        # Passagem Direta - os dados de entrada são passados para o modelo
        outputs = model(input_ids=inputs_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        # A função sigmóide é aplicada aos logits para converter os valores em probabilidades
        logits = outputs.logits
        predicted_probs = sigmoid(logits)

        # A perda é calculada comparando os logits com os rótulos verdadeiros (targets)
        loss = criterion(logits, targets) # função de perda criterion

        # Retropropagação e Otimização
        optimizer.zero_grad()
        loss.backward() #  gradiente é calculado
        optimizer.step() # parâmetros do modelo são otimizados

    # Loop de Validação
    model.eval() # modo de avaliação
    all_targets = []
    all_predictions = []
    with torch.no_grad():
        for batch in val_loader:
            inputs_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            token_type_ids = batch['token_type_ids']
            targets = batch['targets']

            # Passagem Direta (inferência) - da mesma foram que o treinamento
            outputs = model(input_ids=inputs_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

            # A função sigmóide é aplicada aos logits para converter os valores em probabilidades
            logits = outputs.logits
            predicted_probs = sigmoid(logits)

            # Aplicação do threshold
            predicted_labels = (predicted_probs > threshold).float()

            # Conversão dos rótulos verdadeiros e previstos para numpy arrays
            targets_np = targets.cpu().numpy()
            predicted_labels_np = predicted_labels.cpu().numpy()

            # Coleção dos targets e predictions para calculas as métricas
            all_targets.extend(targets_np)
            all_predictions.extend(predicted_labels_np)

    # Calcular as métricas
    f1_macro = f1_score(all_targets, all_predictions, average='macro')
    precision_macro = precision_score(all_targets, all_predictions, average='macro')
    recall_macro = recall_score(all_targets, all_predictions, average='macro')
    f1_micro = f1_score(all_targets, all_predictions, average='micro')
    precision_micro = precision_score(all_targets, all_predictions, average='micro')
    recall_micro = recall_score(all_targets, all_predictions, average='micro')

    # Atualizar a lista das métricas apenas da última epoch
    if epoch == epochs - 1:
        f1_score_list_macro.append(f1_macro)
        precision_list_macro.append(precision_macro)
        recall_list_macro.append(recall_macro)
        f1_score_list_micro.append(f1_micro)
        precision_list_micro.append(precision_micro)
        recall_list_micro.append(recall_micro)

    # Imprimir as métricas ao final de cada época
    print(f"Epoch {epoch + 1}:")
    print(f"F1-Score (macro): {f1_macro:.4f}, Precision (macro): {precision_macro:.4f}, Recall (macro): {recall_macro:.4f}")
    print(f"F1-Score (micro): {f1_micro:.4f}, Precision (micro): {precision_micro:.4f}, Recall (micro): {recall_micro:.4f}")

# Calcular o tempo total de execução
end_time = time.time()
elapsed_time = end_time - start_time
hours, rem = divmod(elapsed_time, 3600)
minutes, seconds = divmod(rem, 60)

print(f"\nTempo de execução: {int(hours):02}:{int(minutes):02}:{seconds:.2f}")

## 5° Fold

In [ ]:
# Dividir o dataset
test_data = df_fold5
train_data = pd.concat([df_fold1, df_fold2, df_fold3, df_fold4], ignore_index=True)

In [ ]:
# Tokenizar o dataset
tokenized_train_data = TokenizeDataset(train_data, tokenizer, max_len)
tokenized_test_data = TokenizeDataset(test_data, tokenizer, max_len)

In [ ]:
# Criar DataLoader para carregar os dados em lotes durante o treinamento
train_loader = DataLoader(tokenized_train_data, batch_size=train_batch_size, shuffle=True)
val_loader = DataLoader(tokenized_test_data, batch_size=valid_batch_size, shuffle=False)

In [ ]:
# Definir modelo, optimizer, e loss function
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=num_labels, problem_type="multi_label_classification")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.BCEWithLogitsLoss()
sigmoid = torch.nn.Sigmoid()

In [ ]:
start_time = time.time()

for epoch in range(epochs):
    # Loop de Treinamento
    model.train() # modo de treinamento
    for batch in train_loader:
        inputs_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        targets = batch['targets']

        # Passagem Direta - os dados de entrada são passados para o modelo
        outputs = model(input_ids=inputs_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        # A função sigmóide é aplicada aos logits para converter os valores em probabilidades
        logits = outputs.logits
        predicted_probs = sigmoid(logits)

        # A perda é calculada comparando os logits com os rótulos verdadeiros (targets)
        loss = criterion(logits, targets) # função de perda criterion

        # Retropropagação e Otimização
        optimizer.zero_grad()
        loss.backward() #  gradiente é calculado
        optimizer.step() # parâmetros do modelo são otimizados

    # Loop de Validação
    model.eval() # modo de avaliação
    all_targets = []
    all_predictions = []
    with torch.no_grad():
        for batch in val_loader:
            inputs_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            token_type_ids = batch['token_type_ids']
            targets = batch['targets']

            # Passagem Direta (inferência) - da mesma foram que o treinamento
            outputs = model(input_ids=inputs_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

            # A função sigmóide é aplicada aos logits para converter os valores em probabilidades
            logits = outputs.logits
            predicted_probs = sigmoid(logits)

            # Aplicação do threshold
            predicted_labels = (predicted_probs > threshold).float()

            # Conversão dos rótulos verdadeiros e previstos para numpy arrays
            targets_np = targets.cpu().numpy()
            predicted_labels_np = predicted_labels.cpu().numpy()

            # Coleção dos targets e predictions para calculas as métricas
            all_targets.extend(targets_np)
            all_predictions.extend(predicted_labels_np)

    # Calcular as métricas
    f1_macro = f1_score(all_targets, all_predictions, average='macro')
    precision_macro = precision_score(all_targets, all_predictions, average='macro')
    recall_macro = recall_score(all_targets, all_predictions, average='macro')
    f1_micro = f1_score(all_targets, all_predictions, average='micro')
    precision_micro = precision_score(all_targets, all_predictions, average='micro')
    recall_micro = recall_score(all_targets, all_predictions, average='micro')

    # Atualizar a lista das métricas apenas da última epoch
    if epoch == epochs - 1:
        f1_score_list_macro.append(f1_macro)
        precision_list_macro.append(precision_macro)
        recall_list_macro.append(recall_macro)
        f1_score_list_micro.append(f1_micro)
        precision_list_micro.append(precision_micro)
        recall_list_micro.append(recall_micro)

    # Imprimir as métricas ao final de cada época
    print(f"Epoch {epoch + 1}:")
    print(f"F1-Score (macro): {f1_macro:.4f}, Precision (macro): {precision_macro:.4f}, Recall (macro): {recall_macro:.4f}")
    print(f"F1-Score (micro): {f1_micro:.4f}, Precision (micro): {precision_micro:.4f}, Recall (micro): {recall_micro:.4f}")

# Calcular o tempo total de execução
end_time = time.time()
elapsed_time = end_time - start_time
hours, rem = divmod(elapsed_time, 3600)
minutes, seconds = divmod(rem, 60)

print(f"\nTempo de execução: {int(hours):02}:{int(minutes):02}:{seconds:.2f}")

## Resultado final (média dos 5 folds)

In [ ]:
# Imprimir as listas de métricas
print("Listas de Métricas:")
print("F1-Score Macro:", f1_score_list_macro)
print("Precision Macro:", precision_list_macro)
print("Recall Macro:", recall_list_macro)
print("F1-Score Micro:", f1_score_list_micro)
print("Precision Micro:", precision_list_micro)
print("Recall Micro:", recall_list_micro)

In [ ]:
# Imprimir as métricas médias obtidas nas 5 partições
print(f"F1-Score Macro médio: {np.mean(f1_score_list_macro):.4f}")
print(f"Precision Macro média: {np.mean(precision_list_macro):.4f}")
print(f"Recall Macro média: {np.mean(recall_list_macro):.4f}")
print(f"F1-Score Micro médio: {np.mean(f1_score_list_micro):.4f}")
print(f"Precision Micro média: {np.mean(precision_list_micro):.4f}")
print(f"Recall Micro média: {np.mean(recall_list_micro):.4f}")